In [1]:
import pandas as pd
import numpy as np


In [2]:
import math

In [3]:
df=pd.read_csv('c:/Users/dibya/Downloads/Compressed/ml-1m/ml-1m/ratings.dat',header=None,sep='::',usecols=[0,1,2],
               names=['userId','movieId','rating'],engine='python')

In [4]:
df.head()

,userId,movieId,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [5]:
df.describe()

,userId,movieId,rating
count,1.000209e+06,1.000209e+06,1.000209e+06
mean,3.024512e+03,1.865540e+03,3.581564e+00
std,1.728413e+03,1.096041e+03,1.117102e+00
min,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.506000e+03,1.030000e+03,3.000000e+00
50%,3.070000e+03,1.835000e+03,4.000000e+00
75%,4.476000e+03,2.770000e+03,4.000000e+00
max,6.040000e+03,3.952000e+03,5.000000e+00


In [6]:
no_movies=max(df['movieId'])+1
no_users=max(df['userId'])+1

In [7]:
no_movies

3953

In [8]:
no_users

6041

In [24]:
formovies=np.zeros(3953)

In [25]:
byuser=np.zeros(no_users)

In [26]:
rated= [set() for i in range(no_users) ]
dictionary={}

In [27]:
import random

In [28]:
for i, row in enumerate(df.itertuples(), 1):
    j=int(row.movieId)
    formovies[j]+=1
    k=int(row.userId)
    byuser[k]+=1
    rated[k].add(j)
    dictionary[(k,j)]=row.Index

In [29]:
thres_for_movie=50

In [30]:
del_rating=0

In [31]:
import time

In [34]:
indices=[]
t1=time.time()
for i in range(no_users):
    
    movies_rated_by_i=rated[i]
    movies_gt_thres=[k for k in movies_rated_by_i if formovies[k]>=thres_for_movie]
    
    length=len(movies_gt_thres)
    if length>=10:
        vals=length-10
        inds = set(random.sample(list(range(len(movies_gt_thres))), vals))
        removed_users = [n for l,n in enumerate(movies_gt_thres) if l in inds]
        for x in removed_users:
            formovies[x]-=1
            ind=dictionary[(i,x)]
            #df.drop(df.index[[ind]],inplace=True)
            indices.append(ind)
        
print(time.time()-t1)    

3.448923349380493


In [35]:
len(indices)

837646

In [36]:
df2=df.drop(df.index[indices])

In [37]:
df2.describe()

,userId,movieId,rating
count,162563.000000,162563.000000,162563.000000
mean,3938.018264,1958.495937,3.385801
std,1751.108008,1142.361862,1.189298
min,1.000000,1.000000,1.000000
25%,2593.000000,1004.000000,3.000000
50%,4389.000000,1994.000000,4.000000
75%,5508.000000,2924.000000,4.000000
max,6040.000000,3952.000000,5.000000


In [38]:
df2.head()

,userId,movieId,rating
4,1,2355,5
10,1,595,5
12,1,2398,4
13,1,2918,4
19,1,2797,4


In [39]:
df2=df2.sample(frac=1).reset_index(drop=True)

In [40]:
df2.head()

,userId,movieId,rating
0,5699,1683,3
1,5404,1124,5
2,4504,1307,2
3,2744,238,1
4,4055,710,2


In [41]:
user_total=np.zeros(no_users)
cnt=np.zeros(no_users,dtype='int64')

In [42]:
for i, row in enumerate(df2.itertuples(), 1):
    k=int(row.userId)
    cnt[k]+=1

In [43]:
import math
test=[math.floor(0.2*i) for i in cnt]

In [44]:
n_test=np.array(test,dtype='int64')
abcd=test
n_train=[j-i for i,j in zip(abcd,cnt)]

In [45]:
tr_temp=pd.DataFrame()
te_temp=pd.DataFrame()

In [46]:
inc_train=np.zeros(no_users,dtype='int64')
inc_hld=np.zeros(no_users,dtype='int64')

In [47]:
dict2={}
rate2= [set() for i in range(no_users) ]
for i, row in enumerate(df2.itertuples(), 1):
    j=int(row.movieId)
    k=int(row.userId)
    rate2[k].add(j)
    dict2[(k,j)]=row.Index

In [48]:
indices=[]
t1=time.time()
for i in range(no_users):
    
    no_rated=cnt[i]
    rated_movies=rate2[i]
    inds = set(random.sample(list(range(len(rated_movies))), int(0.2*len(rated_movies))))
    
    removed_movies = [n for l,n in enumerate(rated_movies) if l in inds]
    
    for x in removed_movies:
        ind=dict2[(i,x)]
        indices.append(ind)
        
print(time.time()-t1)   

0.23633861541748047


In [49]:
train=df2.drop(df2.index[indices])

In [50]:
train.describe()

,userId,movieId,rating
count,131926.000000,131926.000000,131926.000000
mean,3929.997165,1960.463116,3.388513
std,1751.834052,1142.563980,1.187726
min,1.000000,1.000000,1.000000
25%,2579.000000,1006.000000,3.000000
50%,4381.000000,1996.000000,4.000000
75%,5502.000000,2926.000000,4.000000
max,6040.000000,3952.000000,5.000000


In [51]:
test=pd.concat([df2, train]).drop_duplicates(keep=False)

In [52]:
test.describe()

,userId,movieId,rating
count,30637.000000,30637.000000,30637.000000
mean,3972.557920,1950.025068,3.374123
std,1747.586147,1141.471020,1.195994
min,1.000000,1.000000,1.000000
25%,2665.000000,988.000000,3.000000
50%,4443.000000,1981.000000,4.000000
75%,5526.000000,2916.000000,4.000000
max,6040.000000,3952.000000,5.000000


In [53]:
train=train.sample(frac=1).reset_index(drop=True)
test=test.sample(frac=1).reset_index(drop=True)

In [54]:
train.to_csv('F:/ml2trainsparse.csv', sep=',',header=True)


In [55]:
test.to_csv('F:/ml2testsparse.csv', sep=',',header=True)